In [16]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

torch.cuda.device_count()
cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')
device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

In [17]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = 'G:/PracaMagisterska/Dane/Mono_Discret_Audio'
excl_folders = ['TS9','MT2']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=3

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()

# split
split = datasplit.DataSplit(dataset, test_train_split=0.8, val_train_split=0.1, shuffle=True)
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

print('dataset size: ', len(dataset))
print('train set size: ', len(split.train_sampler))
print('val set size: ', len(split.val_sampler))
print('test set size: ', len(split.test_sampler))
dataset.fx_to_label

dataset size:  111072
train set size:  79971
val set size:  8886
test set size:  22215


{'808': 0,
 'BD2': 1,
 'BMF': 2,
 'DPL': 3,
 'DS1': 4,
 'FFC': 5,
 'MGS': 6,
 'OD1': 7,
 'RAT': 8,
 'RBM': 9,
 'SD1': 10,
 'VTB': 11}

In [18]:
# model
input_size = 87  # Update this to match the feature size of your input
hidden_size = 128  # You can adjust this as needed
num_layers = 2  # You can adjust this as needed

fxnet = models.FxRNN(n_classes=dataset.num_fx, input_size=input_size, hidden_size=hidden_size, num_layers=num_layers).to(device)

# optimizer
optimizer_fxnet = optim.Adam(fxnet.parameters(), lr=0.001)
# loss function
loss_func_fxnet = nn.CrossEntropyLoss()

print(fxnet)

FxRNN(
  (lstm): LSTM(87, 128, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=128, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=12, bias=True)
)


In [19]:
# SAVE
models_folder = 'G:/PracaMagisterska/Kod_na_magisterke/gfx-classifier_RNN_version-models_and_results/models'
model_name = '20201210_fxnet_mono_disc_noTS9_best'
results_folder = 'G:/PracaMagisterska/Kod_na_magisterke/gfx-classifier_RNN_version-models_and_results/result'
results_subfolder = '20201210_fxnet_mono_disc_noTS9'

In [20]:
# TRAIN and TEST FxNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

best_val_correct = 0
early_stop_counter = 0

start = time.time()

for epoch in range(100):
    train_loss, train_correct, train_results = trainer.train_fx_net(
        model=fxnet,
        optimizer=optimizer_fxnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_fx_net(
        model=fxnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler, 
        device=device
    )
    
    test_loss, test_correct, test_results = trainer.test_fx_net(
        model=fxnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler, 
        device=device
    )
    
    # Save the best model
    if val_correct > best_val_correct:
        best_val_correct = val_correct
        torch.save(fxnet.state_dict(), f'{models_folder}/{model_name}')
        early_stop_counter = 0
        print('\n=== saved best model ===\n')
    else:
        early_stop_counter += 1
        
    # Append results
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

    if early_stop_counter == 15:
        print('\n--- early stop ---\n')
        break

stop = time.time()
print(f"Training time: {stop - start}s")

Train Epoch: 0	[5000/79971 (6%)]	Total Loss: 119.9477	Avg Loss: 0.0240
Train Epoch: 0	[10000/79971 (12%)]	Total Loss: 228.5768	Avg Loss: 0.0229
Train Epoch: 0	[15000/79971 (19%)]	Total Loss: 332.5237	Avg Loss: 0.0222
Train Epoch: 0	[20000/79971 (25%)]	Total Loss: 432.6240	Avg Loss: 0.0216
Train Epoch: 0	[25000/79971 (31%)]	Total Loss: 531.3028	Avg Loss: 0.0213
Train Epoch: 0	[30000/79971 (38%)]	Total Loss: 627.8387	Avg Loss: 0.0209
Train Epoch: 0	[35000/79971 (44%)]	Total Loss: 722.8787	Avg Loss: 0.0207
Train Epoch: 0	[40000/79971 (50%)]	Total Loss: 817.8710	Avg Loss: 0.0204
Train Epoch: 0	[45000/79971 (56%)]	Total Loss: 910.0089	Avg Loss: 0.0202
Train Epoch: 0	[50000/79971 (62%)]	Total Loss: 999.1285	Avg Loss: 0.0200
Train Epoch: 0	[55000/79971 (69%)]	Total Loss: 1086.1547	Avg Loss: 0.0197
Train Epoch: 0	[60000/79971 (75%)]	Total Loss: 1174.9100	Avg Loss: 0.0196
Train Epoch: 0	[65000/79971 (81%)]	Total Loss: 1258.1261	Avg Loss: 0.0194
Train Epoch: 0	[70000/79971 (88%)]	Total Loss: 134

In [21]:
# BEST RESULTS
print('Accuracy: ', 100 * max(all_train_correct) / train_set_size)
print('Epoch: ', np.argmax(all_train_correct))
print()
print('Accuracy: ', 100 * max(all_val_correct) / val_set_size)
print('Epoch: ', np.argmax(all_val_correct))
print()
print('Accuracy: ', 100 * max(all_test_correct) / test_set_size)
print('Epoch: ', np.argmax(all_test_correct))
print()

Accuracy:  99.44354828625377
Epoch:  82

Accuracy:  98.59329282016655
Epoch:  68

Accuracy:  98.45149673643934
Epoch:  73



In [23]:
# SAVE RESULTS - all losses, all correct, best results
np.save(f'{results_folder}/{results_subfolder}/all_train_losses.npy', np.array(all_train_losses))
np.save(f'{results_folder}/{results_subfolder}/all_train_correct.npy', np.array(all_train_correct))
np.save(f'{results_folder}/{results_subfolder}/best_train_results.npy', np.array(all_train_results[np.argmax(all_train_correct)]))

np.save(f'{results_folder}/{results_subfolder}/all_val_losses.npy', np.array(all_val_losses))
np.save(f'{results_folder}/{results_subfolder}/all_val_correct.npy', np.array(all_val_correct))
np.save(f'{results_folder}/{results_subfolder}/best_val_results.npy', np.array(all_val_results[np.argmax(all_val_correct)]))

np.save(f'{results_folder}/{results_subfolder}/all_test_losses.npy', np.array(all_test_losses))
np.save(f'{results_folder}/{results_subfolder}/all_test_correct.npy', np.array(all_test_correct))
np.save(f'{results_folder}/{results_subfolder}/best_test_results.npy', np.array(all_test_results[np.argmax(all_test_correct)]))

np.save(f'{results_folder}/{results_subfolder}/fx_labels.npy', np.array(list(dataset.fx_to_label.keys())))